### Welcome to Week 5 Day 1

AutoGen AgentChat!

This should look simple and familiar, because it has a lot in common with Crew and OpenAI Agents SDK

In [ ]:
from dotenv import load_dotenv
load_dotenv(override=True)

### First concept: the Model

In [ ]:
# model which is similar to concepts like LLm its like a wrapper around calling a large language model
from autogen_ext.models.openai import OpenAIChatCompletionClient
model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")

In [ ]:
# eample run local model with ollama
from autogen_ext.models.ollama import OllamaChatCompletionClient
ollamamodel_client = OllamaChatCompletionClient(model="llama3.2")

### Second concept: The Message

In [ ]:
# Different concept from Autogen agent chat messages
from autogen_agentchat.messages import TextMessage
message = TextMessage(content="I'd like to go to London", source="user") # 'user' refers to  'me'
message

### Third concept: The Agent

In [ ]:
from autogen_agentchat.agents import AssistantAgent
# most fundamental class working within AUtogen agent chat
agent = AssistantAgent(
    name="airline_agent",
    model_client=model_client, # underlying LLM
    system_message="You are a helpful assistant for an airline. You give short, humorous answers.", # similar to instructions in OpenAI system messages 
    model_client_stream=True # is how we say that we want to stream back the results
)

### Put it all together with on_messages

In [ ]:
# the thing that brings it all together
# that is what we call on an agent to pass in a bunch of messages, put the messages in a list even if its just one message
# also have to pass in a cancellation token, which is how it knows when the messages are finished.
# async as its a couroutine so have to await it
from autogen_core import CancellationToken
response = await agent.on_messages([message], cancellation_token=CancellationToken())
response.chat_message.content

### Let's make a local database of ticket prices

In [ ]:
import os
import sqlite3

# Delete existing database file if it exists
if os.path.exists("tickets.db"):
    os.remove("tickets.db")

# Create the database and the table
conn = sqlite3.connect("tickets.db")
c = conn.cursor()
c.execute("CREATE TABLE cities (city_name TEXT PRIMARY KEY, round_trip_price REAL)")
conn.commit()
conn.close()

In [ ]:
# Populate our database
def save_city_price(city_name, round_trip_price):
    conn = sqlite3.connect("tickets.db")
    c = conn.cursor()
    c.execute("REPLACE INTO cities (city_name, round_trip_price) VALUES (?, ?)", (city_name.lower(), round_trip_price))
    conn.commit()
    conn.close()

# Some cities!
save_city_price("London", 299)
save_city_price("Paris", 399)
save_city_price("Rome", 499)
save_city_price("Madrid", 550)
save_city_price("Barcelona", 580)
save_city_price("Berlin", 525)

In [ ]:
# Method to get price for a city
def get_city_price(city_name: str) -> float | None:
    # didn't have any kind of decorator or anything.
    # In OpenAI agemts SDK we put in a decotatror, in Landgraph we  wrap it in a 'tool'
    # Did not have to do anything like that, really lightweight
    # And it's because, of course, they've just got a little bit of extra stuff in the abstraction code
    # that sees this as a Python function.
    # It uses this comment to figure out the description of the tool.
    # So it just does a bit of that for you.
    # It makes life a tiny bit easier.
    """ Get the roundtrip ticket price to travel to the city """
    conn = sqlite3.connect("tickets.db")
    c = conn.cursor()
    c.execute("SELECT round_trip_price FROM cities WHERE city_name = ?", (city_name.lower(),))
    result = c.fetchone()
    conn.close()
    return result[0] if result else None

In [ ]:
get_city_price("Rome")

In [ ]:
from autogen_agentchat.agents import AssistantAgent

smart_agent = AssistantAgent(
    name="smart_airline_agent",
    model_client=model_client,
    system_message="You are a helpful assistant for an airline. You give short, humorous answers, including the price of a roundtrip ticket.",
    model_client_stream=True,
    tools=[get_city_price], # pass in the tooll functions
    # which is a way of indicating that we don't just want it to return the tool results.
    # we do want it to be able to take that and continue processing even after the tool has returned.
    # Rare to want it false, so should always ssume that  will be your default
    reflect_on_tool_use=True 
)

In [ ]:
response = await smart_agent.on_messages([message], cancellation_token=CancellationToken())
# Um, and then just just for the for fun, I'm going to print the inner messages.
# I mentioned that the message construct isn't just for the human to the agent.
# It's also for what's going on between agents and inside the agent.
# So we can see that by printing its inner messages.
for inner_message in response.inner_messages:
    print(inner_message.content)
response.chat_message.content